In [ ]:

from docuverse.engines.data_template import DataTemplate
from docuverse.utils import read_config_file

In [ ]:
# file1="../output/ibmsw-dev-short-milvus-dense-slate30m-512-100-10062024.cc.json"
# file2="../output/ibmsw-dev-short-milvus-dense-slate30m-512-100-10062024.dd.json"
file1="../output/unified-search-short23k-milvus-dense-granite30m-512-100-20250709-4q.no-rerank.json"
file2="../output/unified-search-short23k-chromadb-granite30m-512-100-20250709-4q.no-rerank.json"
data_template="../config/ibm_search_beir_data.yml"

data_config = read_config_file(data_template)

query_template = DataTemplate(data_config['query_format'])

In [ ]:
from docuverse import SearchEngine

res1 = SearchEngine.read_output_(file1, query_template)
res2 = SearchEngine.read_output_(file2, query_template)

In [ ]:
num_examples = len(res1)
top_k = 5
res = []
for i, (r1, r2) in enumerate(zip(res1, res2)):
    mismatch = False
    mismatched = []
    r1_size = len(r1.retrieved_passages)
    r2_size = len(r2.retrieved_passages)
    for j in range(min(top_k, max(r1_size, r2_size))):
        print(f"i={i}, j={j}")
        if j>=r1_size:
            mismatched.append([[], [r2[j]['id'], r2[j]['score']]])
        elif j>=r2_size:
            mismatched.append([[r1[j]['id'], r1[j]['score']], []])
        else:
            if r1[j]['id'] != r2[j]['id']:
                mismatch = True
                #print(f"Mismatched at question {i}: {r1.question}, result {j}: {r1[j]} vs {r2[j]}")
                mismatched.append([[r1[j]['id'], r1[j]['score']], [r2[j]['id'], r2[j]['score']]])
    res.append(mismatched)

In [ ]:
[r for r in res if len(r)>0]

In [ ]:
res2[2].question.as_dict()

In [ ]:
print(f"r1={res1[0][0]['text']}"
      f"\n\nr2={res2[0][0]['text']}")